In [11]:
import pandas as pd
import numpy as np
from functions import *
from pytorch_util import *
from pytorch_models import ConvBatchLeaky1D,GRU_NCL,LambdaLayer
from torch.nn import Linear,Sequential,LeakyReLU,Dropout,L1Loss
from torch.utils.data import DataLoader
from torch.optim import Adam

In [2]:
train = pd.read_csv("../Data/train.csv", 
                    dtype={'acoustic_data': np.float32, 'time_to_failure': np.float32})
train = train.values
train[:,0] = normalize(train[:,0])
# without tanh is worse-off

In [60]:
## change ##
batch_size = 16
clip = .25

Prepare data

In [4]:
train_gen = SequenceGen(train)
train_gen = DataLoader(train_gen,batch_size,False,num_workers=2)

val_gen = SequenceGen(train,False)
val_gen = DataLoader(val_gen,batch_size,False,num_workers=2)

Build model

In [89]:
model = Sequential(ConvBatchLeaky1D(1,2,5,stride=2),
                      ConvBatchLeaky1D(2,4,5,stride=2),
                      ConvBatchLeaky1D(4,8,5,stride=2),
                      ConvBatchLeaky1D(8,16,5,stride=2),
                      ConvBatchLeaky1D(16,32,5,stride=2),
                      ResidualBlockGLU(32),
                      ResidualBlockGLU(64),
                      Dropout(0.5),
                      ResidualBlockGLU(128),
                      Dropout(0.5),
                      ResidualBlockGLU(256),
                      Dropout(0.5),
                      Linear(291,1),
                      LeakyReLU(0.1,True),
                      LambdaLayer(lambda x:x.squeeze(2)),
                      Dropout(0.5),
                      Linear(512,1),
                      LambdaLayer(lambda x:x.squeeze(1))
                     ).to('cuda:0')

In [90]:
opt = Adam(trainable_parameter(model),lr=1e-3)

In [91]:
loss_func_nor = loss_func_generator(L1Loss())

Training

In [92]:
model = fit(5, model, loss_func_nor, opt, train_gen, val_gen,clip=clip)

epoch:0, train_loss:2.6424664556980133, val_loss:3.00357723236084
epoch:1, train_loss:2.197013653241671, val_loss:2.4177353382110596
epoch:2, train_loss:2.1200337977363515, val_loss:2.5188286304473877
epoch:3, train_loss:2.0404999198821874, val_loss:2.6089489459991455
epoch:4, train_loss:1.9742543247456734, val_loss:2.68023419380188
Training completed in 41.06319046020508s


Submission

In [93]:
submission = pd.read_csv('../Data/sample_submission.csv',)
test_gen = SequenceGenTest(submission.seg_id.tolist())
test_gen = DataLoader(test_gen,batch_size,False,num_workers=2)
submission.iloc[:,1] = predict(model,test_gen)
submission.to_csv('../Submission/sub_raw_linear.csv',index=False)